In [1]:
import os
import itertools

import numpy as np
import wandb
import seaborn as sn

import utils
from naive_model import NaiveModel
import encoding

NUM_EMBEDDING = 2000

In [2]:
import torch

import torch.nn as nn
assert torch.cuda.is_available()

def to_device(d):
    if hasattr(d, 'cuda'):
        return d.cuda()
    return {k: v.cuda() for k, v in d.items()}

class Model(nn.Module):
    def __init__(self, units, combinations=encoding.NAMES):
        super().__init__()
        self.units = units

        self.embed = nn.Embedding(num_embeddings=NUM_EMBEDDING, embedding_dim=units)
        self.lstm = nn.LSTM(input_size=units, hidden_size=units, num_layers=1, batch_first=True, bidirectional=True)

        self.tasks = {}
        for combination in combinations:
            out = nn.Linear(in_features=units, out_features=encoding.class_size(combination))
            self.tasks[combination] = out
            setattr(self, 'x'.join(combination), out)

    def forward(self, x):
        embeds = self.embed(x)

        lstm_out, (h_n, c_n) = self.lstm(embeds)
        left, right = torch.chunk(h_n, 2, dim=0)
        merge = torch.squeeze(left + right)

        outputs = { combination: f(merge) for combination, f in self.tasks.items() }
        return outputs


In [3]:
import functools
import operator as op

def sanity():
    model = Model(100, combinations=[('B', 'T')])
    print(model)
    with torch.no_grad():
        verbs = encoding.wordlist2numpy(["אתאקלם", "יכפיל"])
        labels = {'B': torch.Tensor([3, 5]), 'T': torch.Tensor([2, 4])}
        verbs = torch.from_numpy(verbs).to(torch.int64)
        tag_scores = model(verbs)
        for combination in tag_scores:
            print(combination)
            v = tag_scores[combination]
            c_labels = functools.reduce(op.mul, [labels[k] for k in combination])
            print(f'{v=}')
            print(f'{c_labels=}')
            print(f'{labels=}')
            print()

sanity()

Model(
  (embed): Embedding(2000, 100)
  (lstm): LSTM(100, 100, batch_first=True, bidirectional=True)
  (BxT): Linear(in_features=100, out_features=28, bias=True)
)
('B', 'T')
v=tensor([[-0.0859, -0.2237, -0.1789,  0.2127, -0.1810,  0.0110, -0.2306,  0.2161,
          0.1873,  0.0856, -0.2968,  0.0109,  0.0547,  0.1172,  0.2727, -0.0527,
         -0.1465,  0.1861, -0.1416,  0.2334, -0.3289,  0.1478, -0.1295, -0.1113,
         -0.1698, -0.0929, -0.2468,  0.2675],
        [-0.1443, -0.0158,  0.0617,  0.2430, -0.2969,  0.0045, -0.2208, -0.0239,
          0.2190, -0.0652, -0.3013, -0.1919, -0.0387, -0.0984,  0.2926, -0.0926,
         -0.1417, -0.1280,  0.0831,  0.2554, -0.3005,  0.2697, -0.2298, -0.0811,
         -0.0923, -0.1898, -0.0565,  0.0399]])
c_labels=tensor([ 6., 20.])
labels={'B': tensor([3., 5.]), 'T': tensor([2., 4.])}



In [4]:
import concrete

def load_dataset(file_pat):
    *features_train, verbs_train = concrete.load_dataset(f'{file_pat}_train.tsv')
    *features_test, verbs_test = concrete.load_dataset(f'{file_pat}_test.tsv')
    return ((encoding.wordlist2numpy(verbs_train), encoding.list_of_lists_to_category(features_train)),
            (encoding.wordlist2numpy(verbs_test) , encoding.list_of_lists_to_category(features_test )))

def load_dataset_split(filename, split):
    *features_train, verbs_train = concrete.load_dataset(filename)
    features_test = [t[-split:] for t in features_train]
    verbs_test = verbs_train[-split:]
    del verbs_train[-split:]
    for t in features_train:
        del t[-split:]
    return ((encoding.wordlist2numpy(verbs_train), encoding.list_of_lists_to_category(features_train)),
            (encoding.wordlist2numpy(verbs_test ), encoding.list_of_lists_to_category(features_test )))


In [19]:
BATCH_SIZE = 128

def batch(a):
    ub = a.shape[0] // BATCH_SIZE * BATCH_SIZE
    return to_device(torch.from_numpy(a[:ub]).to(torch.int64)).split(BATCH_SIZE)

def ravel_multi_index(ys, combinations):
    nsamples = len(next(iter(ys.values())))
    return {combination: np.array([
        np.ravel_multi_index(tuple(ys[k][i] for k in combination), encoding.combined_shape(combination))
        for i in range(nsamples)
    ]) for combination in combinations}

def batch_all_ys(ys):
    res = []
    m = {k: batch(ys[k]) for k in ys}
    nbatches = len(next(iter(m.values())))
    for i in range(nbatches):
        res.append({k: m[k][i] for k in ys})
    return res

def fit(model, train, test, *, epochs,  runsize, criterion, optimizer, phases, teacher):
    x_train, y_train = train
    x_test, y_test = test
    data = {
        'train': (batch(x_train), batch_all_ys(y_train)),
        'test':  (batch(x_test ), batch_all_ys(y_test ))
    }

    stats = utils.Stats(model.tasks.keys(), runsize)
    
    for epoch in range(epochs):
        stats.epoch_start()
        
        for phase in phases:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            stats.phase_start(phase, batches_in_phase=len(data[phase][0]))

            for inputs, labels in zip(*data[phase]):
                stats.batch_start()
                
                if phase == 'train':
                    outputs = model(inputs)
                else:
                    with torch.no_grad():
                        outputs = model(inputs)

                if teacher is not None:
                    pseudo_labels = teacher(inputs)
                    losses = {k: criterion(outputs[k].double(), pseudo_labels[k]) for k in outputs}
                else:
                    losses = {combination: criterion(output.double(), labels[combination])
                              for combination, output in outputs.items()}
                
                if phase == 'train' and isinstance(criterion, nn.CrossEntropyLoss):
                    stats.assert_reasonable_initial(losses)
                
                loss = sum(losses.values())

                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                
                stats.update(loss=loss.item(),
                             batch_size=inputs.size(0),
                             d={k: (outputs[k], labels[k].detach()) for k in outputs})
                
                stats.batch_end()
            stats.phase_end()
        stats.epoch_end()
    return stats

@torch.no_grad()
def predict(model, *verbs):
    model.eval()
    verbs = encoding.wordlist2numpy(verbs)
    verbs = to_device(torch.from_numpy(verbs).to(torch.int64))
    outputs = model(verbs)
    res = {}
    # FIX: assumes no overlaps
    for combination, v in outputs.items():
        combined_index = torch.argmax(v).cpu().data.numpy()
        indices = np.unravel_index(combined_index, encoding.combined_shape(combination))
        for k, i in zip(combination, indices):
            assert k not in res, "Overlapping classes are not handled"
            res[k] = encoding.from_category(k, i)
    if all(r in res for r in ['R1', 'R2', 'R3', 'R4']):
        res['R'] = ''.join(res[k] for k in ['R1', 'R2', 'R3', 'R4']).replace('.', '')
    return res
    

In [22]:
arity = '3'
gen = 'all'
artifact_name = f'{gen}_{arity}_shuffled'
filename = f'synthetic/{artifact_name}.tsv'  # all_verbs_shuffled
test_size = 5000

artifact = wandb.Artifact(artifact_name, type='dataset')
artifact.add_file(filename)

(train_x, pre_train_y), (test_x, pre_test_y) = load_dataset_split(filename, split=test_size)

In [8]:
def naive_config(filename):
    return {
        'model': NaiveModel.learn_from_file(filename),
        'phases': ['test'],
        'criterion': nn.CrossEntropyLoss(),
        'optimizer': None
    }

def teacher_config(train):
    res = standard_config()
    res['teacher'] = NaiveModel.learn_from_data(train)
    res['criterion'] = nn.BCEWithLogitsLoss()  # BCELoss: works, but total loss is nan
    return res


def standard_config(model, lr):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    return {
        'model': model,
        'criterion': nn.CrossEntropyLoss(),
        'optimizer': optimizer,
        'phases': ['train', 'test'],
        'teacher': None
    }

In [ ]:
%env WANDB_SILENT true
%env WANDB_MODE dryrun

config = {
    'optimizer': 'adam',
    'batch_size': BATCH_SIZE,
    'epochs': 1,
    'runsize': 8192 // BATCH_SIZE,
    'test_size': test_size,
}
# group = 

def experiment(combinations):
    train_y = ravel_multi_index(pre_train_y, combinations)
    test_y = ravel_multi_index(pre_test_y, combinations)
    train = (train_x, train_y)
    test = (test_x, test_y)
    
    units = 400
    lr = 8e-4

    config.update({
        'units': units,
        'lr': lr,
    })
    names_str = '+'.join('x'.join(combination) for combination in combinations)
    run = wandb.init(project="rootem",
                     group=f'joint-{gen}-{arity}',  # f'lr_units_grid_search-{arity}-{wandb.util.generate_id()}',
                     name=f'joint{gen}-{arity}-{names_str}',  # f'{gen}-{arity}-{lr:.0e}',# f'{arity}-batch_{BATCH_SIZE}', # f'all-{arity}-lr_{lr:.0e}-units_{units}',
                     tags=[gen, arity, 'synthetic', 'shuffle', 'no_prefix'],
                     config=config)

    run.use_artifact(artifact)

    wandb.config.update(config, allow_val_change=True)

    print(config)
    
    model = to_device(Model(units=units, combinations=combinations))
    if isinstance(model, nn.Module):
        wandb.watch(model)
        
    stats = fit(train=train,
        test=test,
        epochs=config['epochs'],
        runsize=config['runsize'],
        **standard_config(model, lr)
    )
    # wandb.save(f"simple_{arity}.h5")
    return model, stats

model, stats = experiment([('B',), ('T',), ('R1', 'R2'), ('R4',)])

env: WANDB_SILENT=true
env: WANDB_MODE=dryrun


{'optimizer': 'adam', 'batch_size': 128, 'epochs': 1, 'runsize': 64, 'test_size': 5000, 'units': 400, 'lr': 0.0008}


In [ ]:
k = 'R1'
labels = [x[::-1] for x in CLASSES[k]]
ax = sn.heatmap(stats.confusion[k], xticklabels=labels, yticklabels=labels, square=True, robust=True, cmap="cividis")

In [ ]:
ax = sn.heatmap(stats.confusion_logprobs[k], xticklabels=labels, yticklabels=labels, square=True, robust=True, cmap="cividis")

In [29]:
print(predict(model, 'סבסו'))
print(predict(model, 'מקדו'))
print(predict(model, 'נמזר'))
print(predict(model, 'כרדו'))

{'B': 'פעל', 'T': 'עבר', 'R1': 'ס', 'R2': 'ב', 'R4': 'ס'}
{'B': 'פעל', 'T': 'עבר', 'R1': 'מ', 'R2': 'ק', 'R4': 'ד'}
{'B': 'פיעל', 'T': 'עתיד', 'R1': 'מ', 'R2': 'ז', 'R4': 'ר'}
{'B': 'פעל', 'T': 'עבר', 'R1': 'כ', 'R2': 'ר', 'R4': 'ד'}


In [21]:
print(predict(model, 'הבריל'))
print(predict(model, 'חגוו'))
print(predict(model, 'עגו'))
print(predict(model, 'צירלל'))

{'B': 'הפעיל', 'T': 'עבר'}
{'B': 'פיעל', 'T': 'עבר'}
{'B': 'פיעל', 'T': 'עבר'}
{'B': 'פיעל', 'T': 'עבר'}


In [ ]:
print(predict(model, "השטקרפתי"))

In [ ]:
print(predict(model, "ישסו"))

In [ ]:
import importlib
import encoding
import naive_model
import utils
encoding = importlib.reload(encoding)
naive_model = importlib.reload(naive_model)
utils = importlib.reload(utils)
wandb = importlib.reload(wandb)
Stats = utils.Stats
NaiveModel = naive_model.NaiveModel

In [ ]:
batch_all_ys(test_y)

In [ ]:
test_y[('B', 'T')].shape

In [ ]:
pre_test_y['B'].shape

In [ ]:
def nonempty_powerset(seq):
    return itertools.chain.from_iterable(itertools.combinations(seq, r) for r in range(1, len(seq)+1))

def tensor_outer_product(a, b, c):
    return torch.einsum('bi,bj,bk->bijk', a, b, c)
